In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import pandas as pd
import jieba
import re

In [13]:
class MyDataset(Dataset):
    def __init__(self, max_len, debug=True):
        super().__init__()
        df = pd.read_csv('../../datasets/THUCNews/train.csv')
        df = df.dropna()
        if debug:
            df = df.sample(2000).reset_index(drop=True)
        else:
            df = df.sample(50000).reset_index(drop=True)
        # 读取常用停用词
        stopwords = [line.strip() for line in open('../../stopwords/cn_stopwords.txt', 'r', encoding='utf-8').readlines()]
        sentences = []
        for title in df['title']:
            # 去除标点符号
            title = re.sub(r'[^\u4e00-\u9fa5]', '', title)
            # jieba分词
            sentence_seged = jieba.cut(title.strip())
            outstr = ''
            for word in sentence_seged:
                if word != '\t' and word not in stopwords:
                    outstr += word
                    outstr += ' '
            if outstr != '':
                sentences.append(outstr)
        # 获取所有词(token), <pad>用来填充不满足max_len的句子
        token_list = ['<pad>'] + list(set(' '.join(sentences).split()))
        # token和index互转字典
        self.token2idx = {token: i for i, token in enumerate(token_list)}
        self.idx2token = {i: token for i, token in enumerate(token_list)}
        self.vocab_size = len(self.token2idx)

        self.inputs = []
        for sentence in sentences:
            tokens = sentence.split()
            input_ = [self.token2idx[token] for token in tokens]
            if len(input_) < max_len:
                self.inputs.append(input_ + [self.token2idx['<pad>']] * (max_len - len(input_)))
            else:
                self.inputs.append(input_[: max_len])

        self.labels = [[label] for label in df['label'].values.tolist()]
        self.n_class = len(df['label'].unique())

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.LongTensor(self.inputs[idx]), torch.LongTensor(self.labels[idx])


class TextBiLSTM(nn.Module):
    def __init__(self, vocab_size, embed_size, n_class, n_hidden):
        super().__init__()
        # 嵌入层
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # rnn层，深度为1
        self.lstm = nn.LSTM(embed_size, n_hidden, 1, batch_first=True, bidirectional=True)
        # 激活函数
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        # 输出头
        self.fc = nn.Linear(n_hidden * 2, n_class)

    def forward(self, x):  # x: [batch_size * 句子长度]
        x = self.embedding(x)  # [batch_size * 句子长度 * embed_size]
        out, (_, _) = self.lstm(x)  # [batch_size * 句子长度 * n_hidden]
        out = self.relu(out)
        out = self.dropout(out)
        logits = self.fc(out[:,-1,:])  # 全连接输出头，[batch_size * n_class]
        return logits

In [16]:
dataset = MyDataset(max_len=10)  # 构造长度为10的句子输入，超过10的句子切掉，不足10的补<pad>
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
model = TextBiLSTM(vocab_size=dataset.vocab_size, embed_size=128,
                   n_class=dataset.n_class, n_hidden=256)
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
for epoch in range(50):
    for feature, target in dataloader:
        optimizer.zero_grad()
        logits = model(feature)
        loss = criterion(logits, target.squeeze())
        loss.backward()
        optimizer.step()
    print('epoch:', epoch + 1, ', loss:', loss.item())

TextBiLSTM(
  (embedding): Embedding(8313, 128)
  (lstm): LSTM(128, 256, batch_first=True, bidirectional=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (fc): Linear(in_features=512, out_features=14, bias=True)
)
epoch: 1 , loss: 2.2370526790618896
epoch: 2 , loss: 1.2687817811965942
epoch: 3 , loss: 1.7982620000839233
epoch: 4 , loss: 1.1352111101150513
epoch: 5 , loss: 1.0432990789413452
epoch: 6 , loss: 0.31441110372543335
epoch: 7 , loss: 0.33986926078796387
epoch: 8 , loss: 0.047737207263708115
epoch: 9 , loss: 0.026677202433347702
epoch: 10 , loss: 0.03607143834233284
epoch: 11 , loss: 0.5002908706665039
epoch: 12 , loss: 0.0061968183144927025
epoch: 13 , loss: 0.00960513111203909
epoch: 14 , loss: 0.003971426747739315
epoch: 15 , loss: 0.002365739783272147
epoch: 16 , loss: 0.0022870022803545
epoch: 17 , loss: 0.003123865695670247
epoch: 18 , loss: 0.0016807448118925095
epoch: 19 , loss: 0.002969244495034218
epoch: 20 , loss: 0.0016908154357224703
epoch: 21 ,

In [17]:
df = pd.read_csv('../../datasets/THUCNews/train.csv')
predict = model(feature).max(1)[1].tolist()
for i in range(len(feature.tolist())):
    print(' '.join([dataset.idx2token[idx] for idx in feature.tolist()[i]]),
          '---> label:',
          dict(zip(df['label'], df['class']))[predict[i]])

教育部 副 部长 章新胜 赞成 文理 分科 图 <pad> <pad> ---> label: 教育
辽足 公布 挂牌 名单 王新欣 上榜 身价 万 成 标王 ---> label: 体育
中国联通 发行 亿 短期 融资券 <pad> <pad> <pad> <pad> <pad> ---> label: 科技
所自 划线 高校 年 理学 考研 分数线 统计 <pad> <pad> ---> label: 教育
高手 分享 雅思 口语 与众不同 <pad> <pad> <pad> <pad> <pad> ---> label: 教育
易建联 迪奥 贝尔 山猫 升级 提单 场分 意义 <pad> <pad> ---> label: 体育
官恩娜 手机 失而复得 保住 男友 密照 短信 图 <pad> <pad> ---> label: 娱乐
福建 福州市 普高 第二轮 投档 降分 录取 <pad> <pad> <pad> ---> label: 教育
受累 高铁 股 半程 公募 冠军 申 赎 正常 <pad> ---> label: 财经
倍 光学 变焦镜头 松下 最新 价元 <pad> <pad> <pad> <pad> ---> label: 科技
揭秘 未婚 女孩 服 避孕药 真的 美容 组图 <pad> <pad> ---> label: 时尚
亚太 股市 升势 暂缓 五大 利好 推股 领涨 <pad> <pad> ---> label: 股票
午评 沪 指 震荡 涨欲攻 板块 全线 飘红 <pad> <pad> ---> label: 股票
海鸥 双色球 期 红蓝 看点 三码定 蓝球 <pad> <pad> <pad> ---> label: 彩票
新闻调查 近三成 住户 采用 地热 方式 采暖 <pad> <pad> <pad> ---> label: 家居
吉尔吉斯 议会 下周 表决 关闭 美军基地 提案 <pad> <pad> <pad> ---> label: 时政
